In [25]:
import pandas as pd
import numpy as np
import re
import os
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import lxml

In [26]:
MAIN_URL = "http://www.law.go.kr/DRF/lawService.do?OC=lyusungwon&type=XML&target=prec&ID="
PATH="/home/lyusungwon/Dropbox/jupyter/LegalProject"
SAVE_PATH = "/home/lyusungwon/Dropbox/jupyter/LegalProject/data"

In [27]:
def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            record[subchild.tag] = subchild.text
        all_records.append(record)
    return pd.DataFrame(all_records)

def crawl_each_panrye(num):
    needs = ["판시사항", "판결요지", "사건명", "참조조문", "판례내용"]
    temp_df = pd.DataFrame()
    case = dict()
    url = MAIN_URL+num
    
    xml = requests.get(url)
    xml.encoding= "utf-8"
    
    soup = BeautifulSoup(xml.text,"xml")
    try:
        case["prncYd"] = num
        for i in needs:
            tem = soup.find(i)
            case[i] = soup.find(i).text.strip()
    except Exception as e:
        print("오류 at : %d"%int(num))
        print(e)
    
    print(num)
    return case



In [29]:
xml_data = open('법령검색목록.xml').read()
list_df = xml2df(xml_data)
crime_df = list_df[list_df["evtKndNm"]=="형사"]
precSeqs = crime_df['precSeq']
crime_df.shape

(18189, 9)

In [30]:
grouped = crime_df.groupby('evtNm').count().reset_index()
grouped.sort_values('curtKndCd', ascending=False)[:20]

,evtNm,curtKndCd,curtKndNm,curtNm,evtKndCd,evtKndNm,evtNo,precSeq,prncYd
2535,사기,383,383,383,383,383,383,383,383
7099,폭력행위등처벌에관한법률위반,233,233,233,233,233,233,233,233
926,공직선거법위반,182,182,182,182,182,182,182,182
1042,교통사고처리특례법위반,179,179,179,179,179,179,179,179
7671,횡령,173,173,173,173,173,173,173,173
2001,배임,146,146,146,146,146,146,146,146
1388,근로기준법위반,140,140,140,140,140,140,140,140
971,관세법위반,135,135,135,135,135,135,135,135
1831,무고,133,133,133,133,133,133,133,133
902,공직선거및선거부정방지법위반,132,132,132,132,132,132,132,132


In [36]:
data_df = pd.DataFrame()
for n, i in enumerate(precSeqs[18000:]):
#     if n%1000 == 0 and data_df is not None:
#         data_df.to_csv(SAVE_PATH+'/crime'+str(n)+'.csv', sep=',', encoding='utf-8')
#         data_df = pd.DataFrame()
    print(n)
    case = crawl_each_panrye(i)
    data_df = data_df.append(case,  ignore_index=True)
data_df.to_csv(SAVE_PATH+'/crime'+str(19000)+'.csv', sep=',', encoding='utf-8')


0
182603
1
182604
2
183079
3
182959
4
183454
5
185511
6
185094
7
185101
8
185613
9
182960
10
185512
11
184941
12
184942
13
184943
14
185100
15
185095
16
182619
17
182620
18
182621
19
182622
20
185513
21
185097
22
182638
23
182639
24
182640
25
182641
26
182654
27
182655
28
182656
29
182657
30
182662
31
182663
32
182664
33
185451
34
185458
35
185459
36
185460
37
185462
38
185463
39
185464
40
185465
41
185466
42
185514
43
182681
44
182683
45
182691
46
182694
47
185461
48
185467
49
185471
50
185515
51
185516
52
184959
53
184960
54
182696
55
185561
56
183805
57
183977
58
184119
59
184121
60
184123
61
183975
62
183066
63
184139
64
183680
65
183681
66
184203
67
184204
68
184206
69
184208
70
184209
71
184210
72
184251
73
184256
74
184257
75
183068
76
183069
77
184040
78
184041
79
184047
80
184048
81
184049
82
184050
83
184146
84
184155
85
184156
86
184157
87
184158
88
183682
89
183976
90
183331
91
184052
92
184053
93
184054
94
184059
95
184061
96
184159
97
184250
98
184380
99
184005
100
183356

In [21]:
data_df.shape

(10, 6)